# Predicting the Adoptability of Shelter Animals: A Classification Analysis

### Group 2
### Members: 
- 

In [1]:
library(readr)
library(tidyverse)
library(ggplot2) 
library(tidymodels)
library(caret)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ purrr     1.0.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.3.0 ──

✔ broom        1.0.7     ✔ rsample      1.2.1
✔ dials        1.4.0     ✔ tune         1.3.0
✔ infer        1.0.7     ✔ workflows    1.2.0
✔ modeldata    1.4.0     ✔ workflowsets 1.1.0
✔ parsnip      1.3.0     ✔ yardstick    1.3.2
✔ recipes      1.1.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::fil

ERROR: Error in library(caret): there is no package called ‘caret’


## Summary

## Introduction

## Methods and Results

*Written explanation:* 

In [2]:
animals <- read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/refs/heads/main/data/2025/2025-03-04/longbeach.csv", show_col_types = FALSE) 
head(animals)

animal_id,animal_name,animal_type,primary_color,secondary_color,sex,dob,intake_date,intake_condition,intake_type,⋯,outcome_date,crossing,jurisdiction,outcome_type,outcome_subtype,latitude,longitude,outcome_is_dead,was_outcome_alive,geopoint
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<date>,<date>,<chr>,<chr>,⋯,<date>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<lgl>,<chr>
A693708,*charlien,dog,white,NA,Female,2013-02-21,2023-02-20,ill mild,stray,⋯,2023-02-26,"[2600 BLK LONG BEACH BLVD, LONG BEACH CA, 90806",Long Beach,euthanasia,ill severe,33.80479,-118.1889,TRUE,FALSE,"33.8047935, -118.1889261"
A708149,NA,reptile,brown,green,Unknown,NA,2023-10-03,normal,stray,⋯,2023-10-03,"`600 BLK E HARCOURT, LB 90805",Long Beach,rescue,other resc,33.86800,-118.2009,FALSE,TRUE,"33.8679994, -118.2009307"
A638068,NA,bird,green,red,Unknown,NA,2020-01-01,injured severe,wildlife,⋯,2020-01-01,"0 BLK GRAND AVE, LONG BEACH, CA 90803",Long Beach,euthanasia,inj severe,33.76048,-118.1481,TRUE,FALSE,"33.7604783, -118.1480912"
A639310,NA,bird,white,gray,Unknown,NA,2020-02-02,ill severe,wildlife,⋯,2020-02-02,"0 BLK TEMPLE AVE, LONG BEACH, CA 90803",Long Beach,transfer,lbah,33.76246,-118.1597,FALSE,TRUE,"33.7624598, -118.1596777"
A618968,*morgan,cat,black,white,Female,2014-12-18,2018-12-18,injured severe,stray,⋯,2019-01-13,"0 BLK W ZANE ST, LONG BEACH, CA 90805",Long Beach,rescue,littlelion,33.84950,-118.1949,FALSE,TRUE,"33.8495009, -118.1949053"
A730385,*brandon,rabbit,black,white,Neutered,2023-04-19,2024-10-18,normal,stray,⋯,2024-11-15,00 AQUARIUM WAY LONG BEACH CA 90802,Long Beach,adoption,web,33.76399,-118.1944,FALSE,TRUE,"33.7639859, -118.1944096"


In [3]:
#cleaning the data: 
animals <- animals %>% 
    select(animal_type, primary_color, sex, dob, intake_condition, intake_type, outcome_type, intake_date, outcome_date) %>% 
    #filter(animal_type != "other") %>%
    na.omit()

animals$outcome_group <- recode(animals$outcome_type,
  "adoption" = "Adopted",
  "foster" = "Adopted",
  "foster to adopt" = "Adopted",
  "homefirst" = "Adopted",
  "rescue" = "Adopted",
  .default = "Not Adopted"  # Everything else is considered "Not Adopted"
)

head(animals)

animal_type,primary_color,sex,dob,intake_condition,intake_type,outcome_type,intake_date,outcome_date,outcome_group
<chr>,<chr>,<chr>,<date>,<chr>,<chr>,<chr>,<date>,<date>,<chr>
dog,white,Female,2013-02-21,ill mild,stray,euthanasia,2023-02-20,2023-02-26,Not Adopted
cat,black,Female,2014-12-18,injured severe,stray,rescue,2018-12-18,2019-01-13,Adopted
rabbit,black,Neutered,2023-04-19,normal,stray,adoption,2024-10-18,2024-11-15,Adopted
other,gray,Unknown,2019-04-12,under age/weight,wildlife,rescue,2019-06-12,2019-06-12,Adopted
cat,black,Unknown,2017-08-21,under age/weight,stray,rescue,2017-09-21,2017-09-23,Adopted
dog,cream,Spayed,2007-12-15,aged,owner surrender,rescue,2024-12-15,2024-12-21,Adopted


In [4]:
#summary of the data: 
summary(animals)
table(animals$outcome_group)
table(animals$outcome_type)
table(animals$intake_condition)

 animal_type        primary_color          sex                 dob            
 Length:26011       Length:26011       Length:26011       Min.   :1993-05-28  
 Class :character   Class :character   Class :character   1st Qu.:2016-07-01  
 Mode  :character   Mode  :character   Mode  :character   Median :2018-10-15  
                                                          Mean   :2018-07-30  
                                                          3rd Qu.:2022-01-12  
                                                          Max.   :2031-03-30  
 intake_condition   intake_type        outcome_type        intake_date        
 Length:26011       Length:26011       Length:26011       Min.   :2017-01-01  
 Class :character   Class :character   Class :character   1st Qu.:2018-07-30  
 Mode  :character   Mode  :character   Mode  :character   Median :2020-06-27  
                                                          Mean   :2020-10-24  
                                                    


    Adopted Not Adopted 
      12327       13684 


               adoption           community cat                    died 
                   6156                     334                     585 
               disposal               duplicate              euthanasia 
                     45                      27                    4148 
                 foster         foster to adopt               homefirst 
                     10                     164                      86 
                missing                  rescue         return to owner 
                     54                    5911                    3107 
       return to rescue  return to wild habitat shelter, neuter, return 
                     46                     146                     896 
               transfer               transport   trap, neuter, release 
                   4128                     133                      35 


              aged     behavior  mild behavior  moderate   behavior  severe 
               155                 68                124                 68 
             feral          fractious         i/i report           ill mild 
               278                634                224               1040 
    ill moderatete         ill severe      injured  mild  injured  moderate 
               759                991                710                780 
   injured  severe         intakeexam             normal   under age/weight 
              1393                  1              12762               5964 
  welfare seizures 
                60 

### Exploratory Analysis Visualizations 

In [ ]:
ggplot(animals, aes(x = animal_type, fill = outcome_type)) +
  geom_bar(position = "fill") +  
  labs(title = "Animal Type vs Outcome Type",
       x = "Animal Type",
       y = "Proportion") +
  theme_minimal() +
  scale_y_continuous(labels = scales::percent)

ggplot(animals, aes(x = animal_type, fill = outcome_group)) +
  geom_bar(position = "fill") + 
  labs(title = "Animal Type vs Outcome Group (adopted / not adopted)",
       x = "Animal Type",
       y = "Proportion") +
  theme_minimal() +
  scale_y_continuous(labels = scales::percent)

In [ ]:
ggplot(animals, aes(x = intake_condition, fill = outcome_type)) +
  geom_bar(position = "fill") + 
  labs(title = "Intake Condition vs Outcome Type",
       x = "Intake Condition",
       y = "Proportion") +
  theme_minimal() +
  scale_y_continuous(labels = scales::percent) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

ggplot(animals, aes(x = intake_condition, fill = outcome_group)) +
  geom_bar(position = "fill") +
  labs(title = "Intake Condition vs Outcome Group (adopted / not adopted)",
       x = "Intake Condition",
       y = "Proportion") +
  theme_minimal() +
  scale_y_continuous(labels = scales::percent) +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))

In [ ]:
animals$age_at_intake <- as.numeric(difftime(Sys.Date(), animals$dob, units = "days")) / 365

# Boxplot of age at intake vs outcome_type
ggplot(animals, aes(x = outcome_group, y = age_at_intake, fill = outcome_group)) +
  geom_boxplot() +
  labs(title = "Age at Intake vs Outcome Group (adopted / not adopted)",
       x = "Outcome Type",
       y = "Age at Intake (Years)") +
  theme_minimal()

### Feature Engineering

In [ ]:
animals$dob <- as.Date(animals$dob)
animals$intake_date <- as.Date(animals$intake_date)
animals$outcome_date <- as.Date(animals$outcome_date)

animals <- animals |> 
  mutate(days_in_shelter = outcome_date - intake_date, age_in_days = outcome_date - dob )

Since studies have shown that the longer animals have been in a shelter, the less likely they are to be adopted. In order to utilize this knowledge as a feature we have created the ```days_in_shelter``` feature by finding the number of days they were in the shelter from their intake day to their outcome day. 

Additionally, many pets are more likely to be adopted when they are young, with older animals being less likely to get adopted. If we were to just use their date of birth as the predictor it would not actually account for how old the animal is was when the left the shelter. To better represent that age we have created the ```age_in_days``` feature, which calculates the number of days old the animal was one their outcome day using ```dob``` and ```outcome_date```.

In [ ]:
animals <- animals |> 
  mutate(age_in_days = as.numeric(age_in_days), days_in_shelter = as.numeric(days_in_shelter)) |> 
  select('animal_type', 'sex', 'intake_condition', 'intake_type', 'days_in_shelter', 'age_in_days', 'outcome_group')
head(animals)

### Classification Analysis 

We currently have one large data set that contains all of our animal shelter observations. In order to be able to gauge the performance of our classification model we need to split it into two data set: training data and testing data.

We will use the training data (```train_data```) to build our model and tune our model and save our testing data (```test_data```) to test the final accuracy of the model on new, unseen data. This will give us an estimate of how the model would perform give unclassified, 'real-world' data.

In [ ]:
# Splitting data into train and test data
set.seed(2)
data_split <- initial_split(animals, prop = 0.7, strata = outcome_group)
train_data <- training(data_split)
test_data <- testing(data_split)
head(train_data)

Now we have to build our classification model. 

We are using **K-Nearest Neighbors** as our classification model as it doesn't have an specific assumption about data distribution, it is robust to noise data, it is easy to implement and tune, and it is an efficient model since it does not require a training phase. 

To tune our model, we are finding the accuracy of the model using different numbers of neighbors (k) from 1 to 25. The higher the k value, the more complex our model becomes, so it is important to ensure that the k is not too high and causing the model to over fit to our training data, and not be general enough to accurately predict the training data.

In [ ]:
recipe <- recipe(outcome_group ~ ., data = train_data) %>%
  step_novel(all_nominal_predictors()) %>%  # Handle new levels
  step_dummy(all_nominal_predictors()) %>%  # Convert categorical to dummy variables
  step_normalize(days_in_shelter, age_in_days)        # Normalize the numerical predictor

knn_spec <- nearest_neighbor(weight_func = 'rectangular', neighbors = tune()) |> 
  set_engine('kknn') |> 
  set_mode('classification')

shelter_vfold <- vfold_cv(train_data, v = 5, strata = outcome_group)

possible_k <- tibble(neighbors = seq(from = 1, to = 10))

knn_workflow <- workflow() |> 
  add_recipe(recipe) |> 
  add_model(knn_spec) |> 
  tune_grid(resamples = shelter_vfold, grid = possible_k) |> 
  collect_metrics()

accuracies <- knn_workflow |>
  filter(.metric == "accuracy")
head(accuracies)

#graph results to find the k value that gives the best accuracy
accuracy_versus_k<-ggplot(accuracies, aes (x=neighbors, y=mean))+
  geom_point()+
  geom_line()+
  labs(x= "Neighbors", y= "Accuracy Estimate")+
  scale_x_continuous(breaks=seq(0,25, by=1))+
  scale_y_continuous(limits=c(0.4, 1.0))
accuracy_versus_k

Using the elbow graph above, we found that the best number of neighbors (k) for our classification is 5. This is the point where any significant improvement in accuracy begins to plateau and choosing a k value that is larger could potentially lead to over fitting the model.

In [ ]:
knn_model <-nearest_neighbor(weight_func = 'rectangular', neighbors = 5) |> 
  set_engine('kknn') |> 
  set_mode('classification')

knn_fit <- workflow() |> 
  add_recipe(recipe) |> 
  add_model(knn_model) |> 
  fit(data = train_data)

predictions <- predict(knn_fit, new_data = test_data) |> 
  bind_cols(test_data)

predictions <- predictions %>%
  mutate(outcome_group = factor(outcome_group))

confusion_matrix <- conf_mat(predictions, truth = outcome_group, estimate = .pred_class)
print(confusion_matrix)

accuracy <- accuracy(predictions, truth = outcome_group, estimate = .pred_class)
print(accuracy)

In [ ]:
importance <- varImp(knn_model, scale = FALSE)

# Convert importance to a data frame
importance_df <- as.data.frame(importance$importance)
importance_df$Feature <- rownames(importance_df)

# Plot feature importance
ggplot(importance_df, aes(x = reorder(Feature, Overall), y = Overall)) +
  geom_bar(stat = "identity") +
  coord_flip() +
  labs(title = "Feature Importance for KNN Model", x = "Features", y = "Importance Score")

## Discussion

## References